###  Implement of Optimus pipeline
#### Pipeline version v6.0.0 with output in h5ad format

#### 1. The wld files used in this pipeline are downloaded from the following link:

##### https://github.com/broadinstitute/warp/blob/develop/pipelines/skylab/optimus/Optimus.wdl
##### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/H5adUtils.wdl
##### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/FastqProcessing.wdl
##### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/StarAlign.wdl
##### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/Metrics.wdl
##### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/RunEmptyDrops.wdl
##### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/CheckInputs.wdl
##### https://github.com/broadinstitute/warp/blob/develop/tasks/skylab/MergeSortBam.wdl

#### recent update information
##### Optimus.wdl: Aug 31, 2023 pipeline_version = "6.0.0"
##### H5adUtils.wdl: Aug 28, 2023
##### StarAlign.wdl: Aug 28, 2023
##### Metrics.wdl: Aug 31, 2023

#### 2. Check the difference between the wdl files in pipeline_version 5.7.4 and 6.0.0

In [1]:
### current directory ###
!pwd

/Users/xiliu/Documents/analysis/terraPipelines/scAtlas/notebook


In [1]:
!diff ../optimus/Optimus_v6.0.0.wdl  ../optimus/Optimus.wdl

6a7
> import "../tasks/LoomUtils.wdl" as LoomUtils
9d9
< import "../tasks/H5adUtils.wdl" as H5adUtils
11a12
> 
13c14
<     description: "The optimus 3' pipeline processes 10x genomics sequencing data based on the v2 chemistry. It corrects cell barcodes and UMIs, aligns reads, marks duplicates, and returns data as alignments in BAM format and as counts in sparse matrix exchange format."
---
>     description: "The optimus 3' pipeline processes 10x genomics sequencing data based on v2 or v3 chemistry. It corrects cell barcodes and UMIs, aligns reads, marks duplicates, and returns data as alignments in BAM format and as counts in sparse matrix exchange format."
40,42d40
<     # read_structure is based on v2 or v3 chemistry
<     String read_struct = checkOptimusInput.read_struct_out
< 
53,54c51,52
<     # Set to Forward, Reverse, or Unstranded to account for stranded library preparations (per STARsolo documentation)
<     String star_strand_mode = "Forward"
---
>     # Set to true to coun

In [3]:
!diff ../tasks/checkInputs_v6.0.0.wdl ../tasks/checkInputs.wdl 

111c111
<     # Check for chemistry version to produce read structure and whitelist
---
>     
116d115
<       echo 16C10M > read_struct.txt
121d119
<       echo 16C12M > read_struct.txt
153d150
<     String read_struct_out = read_string("read_struct.txt")


In [2]:
!diff ../tasks/FastqProcessing_v6.0.0.wdl ../tasks/FastqProcessing.wdl 

11d10
<     String read_struct
14c13
<     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.1-1686932671"
---
>     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.1-1679490798"
33d31
<     read_struct: "read structure for the 10x chemistry. This automatically selected in the checkInputs task"
105a104,105
>         --barcode-length 16 \
>         --umi-length $UMILENGTH \
109d108
<         --read-structure "~{read_struct}" \
140c139
<     String docker =  "us.gcr.io/broad-gotc-prod/warp-tools:1.0.1-1686932671"
---
>     String docker =  "quay.io/humancellatlas/secondary-analysis-sctools:v0.3.14-test2"
234,261d232
< 
< task FastqProcessATAC {
< 
<     input {
<         Array[String] read1_fastq
<         Array[String] read3_fastq
<         Array[String] barcodes_fastq
<         String read_structure = "16C"
<         String barcode_orientation = "FIRST_BP_RC"
<         String output_base_name
<         File whitelist
< 
<         # [?] copied from corresponding optimu

In [4]:
!diff ../tasks/Metrics_v6.0.0.wdl ../tasks/Metrics.wdl

12c12
<     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.6-1692962087"
---
>     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.1-1679941323"
62c62,63
<     mv updated.~{input_id}.cell-metrics.csv ~{input_id}.cell-metrics.csv
---
>     # remove the following columns: reads_unmapped, reads_mapped_exonic, reads_mapped_intronic, reads_mapped_utr, duplicate_reads, reads_mapped_intergenic
>     cut -d',' -f 1-4,8-9,11-26,29-36 updated.~{input_id}.cell-metrics.csv > ~{input_id}.cell-metrics.csv
88c89
<     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.5-1692706846"
---
>     String docker = "us.gcr.io/broad-gotc-prod/warp-tools:1.0.1-1679941323"
128c129,130
<     mv updated.~{input_id}.gene-metrics.csv ~{input_id}.gene-metrics.csv
---
>     # remove the following columns: reads_mapped_exonic, reads_mapped_intronic, reads_mapped_utr, duplicate_reads
>     cut -d',' -f 1-4,8-9,11-27 updated.~{input_id}.gene-metrics.csv > ~{input_id}.gene-metrics.csv
265c267,

In [5]:
!diff ../tasks/StarAlign_v6.0.0.wdl ../tasks/StarAlign.wdl

222d221
<     String star_strand_mode
228c227
<     String docker = "us.gcr.io/broad-gotc-prod/star:1.0.1-2.7.11a-1692706072"
---
>     String docker = "us.gcr.io/broad-gotc-prod/star:1.0.0-2.7.9a-1658781884"
245d243
<     star_strand_mode: "STAR mode for handling stranded reads. Options are 'Forward', 'Reverse, or 'Unstranded'"
281c279
<         COUNTING_MODE="GeneFull_Ex50pAS"
---
>         COUNTING_MODE="GeneFull"
286,294d283
< # Check that the star strand mode matches STARsolo aligner options
<     if [[ "~{star_strand_mode}" == "Forward" ]] || [[ "~{star_strand_mode}" == "Reverse" ]] || [[ "~{star_strand_mode}" == "Unstranded" ]]
<     then
<         ## single cell or whole cell
<         echo STAR mode is assigned
<     else
<         echo Error: unknown STAR strand mode: "~{star_strand_mode}". Should be Forward, Reverse, or Unstranded.
<         exit 1;
<     fi
307c296
<       --soloStrand ~{star_strand_mode} \
---
>       --soloStrand Unstranded \
320,322c309,310
<       --out

In [7]:
!diff ../tasks/MergeSortBam_v6.0.0.wdl ../tasks/MergeSortBam.wdl

In [9]:
!diff ../tasks/RunEmptyDrops_v6.0.0.wdl ../tasks/RunEmptyDrops.wdl

#### 3. Pipeline validation

In [ ]:
### Syntex validation ###
!java -jar ~/womtool/womtool-85.jar validate -l ~/terraPipelines/scAtlas/Optimus_v6.0.0.wdl